In [2]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import jsonlines

In [28]:
page_url = lambda i: f"https://shop2banh.vn/nhot-xe-may/trang-{i}.html"

In [29]:
def get_list_product_links(page_url):
    try:
        req = requests.get(page_url)
        soup = BeautifulSoup(req.text, 'html.parser')
        # get div by class `category-products products`
        div = soup.find('div', class_='items-list-prod')
        # find all dive with class `product-info`
        products = div.find_all('div', class_='title-prod aaccc')
        # get links of products
        product_links = [product.find('a')['href'] for product in products]
    except:
        product_links = []
    return product_links

In [30]:
def get_product_info(product_link):
    try:
        req = requests.get(product_link)
        soup = BeautifulSoup(req.text, 'html.parser')
        # title-product as div
        title = soup.find('div', class_='col-lg-7 col-md-7 col-sm-12 col-xs-12 right-detail').find('h1').text.strip()
        # price product-price as span
        price = soup.find('span', class_='ct-df').text.strip()
        # get img link
        div = soup.find('div', class_='owl-carousel owl-theme')
        img_divs = div.find_all('div', class_='item')
        img_links = [img_div.find('img')['data-src'] for img_div in img_divs]
    except:
        title = ''
        price = ''
        img_links = []

    return title, price, img_links

In [31]:
page_no = 1
product_links = []

while True:
    links = get_list_product_links(page_url(page_no))
    if not links:
        break
    for link in links:
        product_links.append(link)
    page_no += 1
product_links

['https://shop2banh.vn/goi-bao-duong-xe-tay-ga-nang-cao-15-buoc-2278.html',
 'https://shop2banh.vn/goi-bao-duong-xe-so-xe-con-tay-nang-cao-15-buoc-2277.html',
 'https://shop2banh.vn/goi-bao-duong-xe-tay-ga-tieu-chuan-11-buoc-2276.html',
 'https://shop2banh.vn/goi-bao-duong-xe-may-so-xe-con-tay-tieu-chuan-11-buoc-2275.html',
 'https://shop2banh.vn/nhot-motul-moto-20w50-1l-2274.html',
 'https://shop2banh.vn/nhot-lap-hop-so-wolver-gl-5-80w90-2273.html',
 'https://shop2banh.vn/nhot-mobil-1-0w40-533.html',
 'https://shop2banh.vn/nhot-mobil-1-gold-5w30-531.html',
 'https://shop2banh.vn/nhot-fuchs-silkolene-pro-4-10w40-xp-1064.html',
 'https://shop2banh.vn/motul-300v-factory-line-10w40-1l-17.html',
 'https://shop2banh.vn/motul-h-tech-100-4t-10w40-18.html',
 'https://shop2banh.vn/motul-7100-10w50-4t-1l-354.html',
 'https://shop2banh.vn/motul-scooter-power-le-5w40-08l-34.html',
 'https://shop2banh.vn/nhot-wolver-racing-synthetic-10w40-1lit-492.html',
 'https://shop2banh.vn/nhot-liqui-moly-4t-sy

In [32]:
len(product_links)

104

In [33]:
product_info = []
for link in tqdm(product_links):
    title, price, img_links = get_product_info(link)
    product_info.append({'title': title, 'price': price, 'img_links': img_links})

100%|██████████| 104/104 [01:04<00:00,  1.62it/s]


In [34]:
# write to file jsonlines
with jsonlines.open('shop2banh.jsonl', mode='w') as writer:
    writer.write_all(product_info)